In [1]:
import os
import requests
cred_url = os.environ["QCLOUD_CONTAINER_INSTANCE_CREDENTIALS_URL"]
r = requests.get(cred_url)
secretId = r.json()["TmpSecretId"]
secretKey = r.json()["TmpSecretKey"]
token = r.json()["Token"]

In [4]:
import os
from qcloud_cos import CosConfig
from qcloud_cos import CosS3Client
from ti.utils import get_temporary_secret_and_token

#### 指定本地文件路径，可根据需要修改。
local_file = "/home/tione/notebook/data/train_final_626.zip"

#### 用户的存储桶，修改为存放所需数据文件的存储桶，存储桶获取参考腾讯云对象存储
bucket="hc-02-1258788535"

#### 用户的数据，修改为对应的数据文件路径，文件路径获取参考腾讯云对象存储
data_key="train_final_626.zip"

#### 获取用户临时密钥
secret_id, secret_key, token = get_temporary_secret_and_token()
config = CosConfig(Region=os.environ.get('REGION'), SecretId=secret_id, SecretKey=secret_key, Token=token, Scheme='https')
client = CosS3Client(config)

####  获取文件到本地
response = client.get_object(
    Bucket=bucket,
    Key=data_key,
)
response['Body'].get_stream_to_file(local_file)

In [1]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm 
import datetime, time 
from utils import distance, haversine, standard, pad_seq 
from scipy.stats import skew, kurtosis
from zipfile import ZipFile
from collections import Counter 

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
%%time

myzip=ZipFile('data/train_final_626.zip')
f=myzip.open('train_final_626.csv')
train_df=pd.read_csv(f)
print(train_df)
f.close()
myzip.close() 

train_df.drop_duplicates(['loadingOrder', 'label'], inplace=True)
test_df = pd.read_csv("data/test_create_final.csv")

del_cols = ['arrive_time_index', 'arrive_time', 'direction', 'geo_hash3', 'selcte_index']
train_df.drop(del_cols, axis=1, inplace=True) 

columns = ['TRANSPORT_TRACE', 'begin_port_name', 'begin_port_position',
        'begin_port_position_hash3', 'begin_port_position_hash4',
        'begin_port_position_hash5', 'carrierName', 'end_port_name',
        'end_port_position', 'end_port_position_hash3',
        'end_port_position_hash4', 'end_port_position_hash5', 'loadingOrder',
        'vesselMMSI', 'test_index', 'geo_hash3', 'geo_hash5',
        'geo_hash4', 'speed', 'longitude',
        'latitude', 'timestamp', 'direction', 'label']

train_df.columns = columns

                TRANSPORT_TRACE               arrive_time  arrive_time_index  \
0                   CNSHK-AEJEA  2020-02-07T17:48:03.000Z              10506   
1                   CNSHK-AEJEA  2020-02-07T17:48:03.000Z              10506   
2                   CNSHK-AEJEA  2020-02-07T17:48:03.000Z              10506   
3                   CNSHK-AEJEA  2020-02-07T17:48:03.000Z              10506   
4                   CNSHK-AEJEA  2020-02-07T17:48:03.000Z              10506   
...                         ...                       ...                ...   
146130  CNSHK-MYTPP-MUPLU-ZADUR  2019-11-25T06:45:05.000Z               2466   
146131  CNSHK-MYTPP-MUPLU-ZADUR  2019-11-25T06:45:05.000Z               2466   
146132  CNSHK-MYTPP-MUPLU-ZADUR  2019-11-25T06:45:05.000Z               2466   
146133  CNSHK-MYTPP-MUPLU-ZADUR  2019-11-25T06:45:05.000Z               2466   
146134  CNSHK-MYTPP-MUPLU-ZADUR  2019-11-25T06:45:05.000Z               2466   

       begin_port_name           begin_

In [3]:
train_df.head(1)

,TRANSPORT_TRACE,begin_port_name,begin_port_position,begin_port_position_hash3,begin_port_position_hash4,begin_port_position_hash5,carrierName,end_port_name,end_port_position,end_port_position_hash3,...,test_index,geo_hash3,geo_hash5,geo_hash4,speed,longitude,latitude,timestamp,direction,label
0,CNSHK-AEJEA,CNSHK,113.86305800000001 22.559462,ws0,ws0b,ws0br,OYSCFP,AEJEA,55.04979 25.022073000000002,thr,...,225,web web web web web web web web web web,webzx webzx webzx webzr webzr webzr webzr webz...,webz webz webz webz webz webz webz webz webz webz,11 10 10 7 7 9 11 15 21 22,113.88364299999999 113.885625 113.885680000000...,22.437238 22.418201999999997 22.417943 22.4042...,2020-01-24T05:38:21.000Z 2020-01-24T05:48:51.0...,19100 16700 16700 17100 16500 16500 16600 1600...,347.458611


In [4]:
test_df.head(1)

,loadingOrder,timestamp,direction,speed,TRANSPORT_TRACE,carrierName,vesselMMSI,longitude,latitude,geo_hash5,...,begin_port_name,end_port_name,begin_port_position,end_port_position,begin_port_position_hash4,begin_port_position_hash5,begin_port_position_hash3,end_port_position_hash4,end_port_position_hash5,end_port_position_hash3
0,AE223035353902,2019-07-03T21:16:48.000Z 2019-07-03T21:34:48.0...,29070.0 34550.0 30160.0 33540.0 35140.0 35300....,24 25 32 33 36 37 40 40 42 43 42 41 40 40 39 3...,CNYTN-MXZLO,OIEQNT,C2075927370,120.09385800000001 120.035707 119.9818 119.970...,22.58132 22.617522 22.658465 22.668688 22.7568...,wsj2k wsj2e wsj2f wsj2f wsj33 wsj3c wsj7b wsj7...,...,CNYTN,MXZLO,114.275347 22.5777,-104.305571 19.085960999999998,ws12,ws122,ws1,9emu,9emud,9em


In [5]:
%%time

def get_stats(values):   # 统计特征
    xlen = len(values)
    vmax = np.max(values)
    vmin = np.min(values)
    return [np.mean(values), vmax, vmin, np.std(values), vmax-vmin,
            np.sum(values), np.median(values), np.count_nonzero(values), len(set(values)), 
            np.count_nonzero(values)/xlen, len(set(values))/xlen, 
            # np.percentile(values, 25), np.percentile(values, 75), 
            skew(values), kurtosis(values), 
            sorted(Counter(values).items(), key=lambda x: x[1], reverse=True)[0][0]]

def feature_engineering(df, is_train=True):
    numerical_fea = []
    categorical_fea = []
    if is_train:
        label = []
    for idx in tqdm(range(df.shape[0])):
        line = df.iloc[idx]

        timestamp = [datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.000Z") for x in line['timestamp'].split(' ')]
        
        # 时间特征
        timestamp_hours = np.array([(x-timestamp[0]).total_seconds()/3600.0 for x in timestamp])  # 减去最初的时间
        timestamp_diff1 = np.diff(timestamp_hours, prepend=timestamp_hours[0])    # 时间一阶差分
        # print(timestamp_diff1[:20])
        xlen = len(timestamp_hours)   # 长度

        begin_port = [float(x) for x in line['begin_port_position'].split(' ')]
        end_port = [float(x) for x in line['end_port_position'].split(' ')]

        # 原始的四个特征
        lon = [float(x) for x in line['longitude'].split(' ')]
        lat  = [float(x) for x in line['latitude'].split(' ')]
        speed = [float(x) for x in line['speed'].split(' ')]
        direction = [float(x) for x in line['direction'].split(' ')]
        lon_diff1 = np.diff(lon, prepend=lon[0])
        lat_diff1 = np.diff(lat, prepend=lat[0])
        speed_diff1 = np.diff(speed, prepend=speed[0])
        direction_diff1 = np.diff(direction, prepend=direction[0])

        # 类别特征
        carrierName = line['carrierName']
        vesselMMSI = line['vesselMMSI']
        begin_hash3, begin_hash4, begin_hash5 = line['begin_port_position_hash3'], line['begin_port_position_hash4'], line['begin_port_position_hash5']
        end_hash3, end_hash4, end_hash5 = line['end_port_position_hash3'], line['end_port_position_hash4'], line['end_port_position_hash5']
        first_hash3, first_hash4, first_hash5 = line['geo_hash3'].split(' ')[0], line['geo_hash4'].split(' ')[0], line['geo_hash5'].split(' ')[0]
        last_hash3, last_hash4, last_hash5 = line['geo_hash3'].split(' ')[-1], line['geo_hash4'].split(' ')[-1], line['geo_hash5'].split(' ')[-1]
        first_year, first_month, first_day = timestamp[0].year, timestamp[0].month, timestamp[0].day
        last_year, last_month, last_day = timestamp[-1].year, timestamp[-1].month, timestamp[-1].day

        # 距离特征
        begin_distance = [distance(begin_port[0], begin_port[1], lon[i], lat[i]) for i in range(xlen)]  # 与起始距离
        begin_dis_cumsum = np.cumsum(begin_distance)   # 累积距离
        begin_dis_diff1 = np.diff(begin_distance, prepend=begin_distance[0])   # 一阶差分距离

        end_distance = [distance(end_port[0], end_port[1], lon[i], lat[i]) for i in range(xlen)]    # 与终止距离
        end_dis_cumsum = np.cumsum(end_distance)    # 累积距离
        end_dis_diff1 = np.diff(end_distance, prepend=end_distance[0]) 

        begin_haversine = [haversine(begin_port[0], begin_port[1], lon[i], lat[i]) for i in range(xlen)] 
        begin_hav_cumsum = np.cumsum(begin_haversine)   # 累积距离
        begin_hav_diff1 = np.diff(begin_haversine, prepend=begin_haversine[0])   # 一阶差分距离

        end_haversine = [haversine(end_port[0], end_port[1], lon[i], lat[i]) for i in range(xlen)]
        end_hav_cumsum = np.cumsum(end_haversine)   # 累积距离
        end_hav_diff1 = np.diff(end_haversine, prepend=end_haversine[0])   # 一阶差分距离

        # 速度特征
        timestamp_diff1_tmp = timestamp_diff1.copy()
        timestamp_diff1_tmp[timestamp_diff1_tmp==0] = np.mean(timestamp_diff1_tmp)
        diff_speed = begin_hav_diff1 / (timestamp_diff1_tmp + 1e-8)    # 差分路程 / 差分时间=速度
        
        timestamp_hours_tmp = timestamp_hours.copy()
        timestamp_hours_tmp[timestamp_hours_tmp == 0] = np.mean(timestamp_hours_tmp)
        cumsum_speed = begin_hav_cumsum / (timestamp_hours_tmp + 1e-8)  # 累积路程 / 累积时间

        # 简单数值特征
        speed_0_nums = (np.array(speed) == 0).sum()
        speed_0_rate = speed_0_nums / xlen 
        speed_0_time = timestamp_diff1[np.array(speed) == 0].sum()
        hash3_nunique = len(set(line['geo_hash3'].split(' ')))
        hash4_nunique = len(set(line['geo_hash4'].split(' ')))
        hash5_nunique = len(set(line['geo_hash5'].split(' ')))
        hash3_rate, hash4_rate, hash5_rate = hash3_nunique/xlen,  hash4_nunique/xlen, hash5_nunique/xlen 
        first_begin_dis, first_begin_hav = distance(begin_port[0], begin_port[1], lon[0], lat[0]), haversine(begin_port[0], begin_port[1], lon[0], lat[0])
        last_begin_dis, last_begin_hav = distance(begin_port[0], begin_port[1], lon[-1], lat[-1]), haversine(begin_port[0], begin_port[1], lon[-1], lat[-1])
        first_end_dis, first_end_hav = distance(end_port[0], end_port[1], lon[0], lat[0]), haversine(end_port[0], end_port[1], lon[0], lat[0])
        last_end_dis, last_end_hav = distance(end_port[0], end_port[1], lon[-1], lat[-1]), haversine(end_port[0], end_port[1], lon[-1], lat[-1])
        fbd_lbd, fbd_lbd_rate = last_begin_dis/first_begin_dis, last_begin_dis-first_begin_dis
        fbh_lbh, fbh_lbh_rate = last_begin_hav/first_begin_hav, last_begin_hav-first_begin_hav
        fed_led, fed_led_rate = first_end_dis/last_end_dis, first_end_dis-last_end_dis
        fed_leh, fed_leh_rate = first_end_hav/last_end_hav, first_end_hav-last_end_hav


        # 正式添加统计特征
        num_feas1 = get_stats(timestamp_hours) + get_stats(timestamp_diff1) + get_stats(lon) + get_stats(lat) + get_stats(speed) + get_stats(direction)
        num_feas2 = get_stats(begin_distance)+get_stats(begin_dis_cumsum)+get_stats(begin_dis_diff1)+get_stats(end_distance)+get_stats(end_dis_cumsum)+get_stats(end_dis_diff1)
        num_feas3 = get_stats(begin_haversine)+get_stats(begin_hav_cumsum)+get_stats(begin_hav_diff1)+get_stats(end_haversine)+get_stats(end_hav_cumsum)+get_stats(end_hav_diff1)
        num_feas4 = get_stats(diff_speed) + get_stats(cumsum_speed)
        num_feas5 = get_stats(lon_diff1) + get_stats(lat_diff1) + get_stats(speed_diff1) + get_stats(direction_diff1)
        num_feas6 = [xlen, speed_0_nums, speed_0_rate, speed_0_time, hash3_nunique, hash4_nunique, hash5_nunique, 
                     hash3_rate, hash4_rate, hash5_rate, first_begin_dis, first_begin_hav,  last_begin_dis, last_begin_hav,
                     first_end_dis, first_end_hav, last_end_dis, last_end_hav, fbd_lbd, fbd_lbd_rate, 
                     fbh_lbh, fbh_lbh_rate, fed_led, fed_led_rate, fed_leh, fed_leh_rate]
        # print(len(num_feas1+num_feas2+num_feas3))
        # break 

        # 添加类别型特征
        cat_feas1 = [carrierName, vesselMMSI, begin_hash3, begin_hash4, begin_hash5, 
                     end_hash3, end_hash4, end_hash5, first_hash3, first_hash4, first_hash5, 
                     last_hash3, last_hash4, last_hash5, first_year, first_month, first_day, 
                     last_year, last_month, last_day]
        
        numerical_fea.append(num_feas1 + num_feas2 + num_feas3 + num_feas4 + num_feas5 + num_feas6)
        categorical_fea.append(cat_feas1)
        if is_train:
            label.append(line['label'])
    
    numerical_fea = np.array(numerical_fea)
    categorical_fea = np.array(categorical_fea)
    # print(numerical_fea.shape)
    num_cols = ['num_{}'.format(str(i)) for i in range(numerical_fea.shape[1])]
    cat_cols = ['cat_{}'.format(str(i)) for i in range(categorical_fea.shape[1])]
    num_df = pd.DataFrame(numerical_fea)
    print(num_df.shape)
    cat_df = pd.DataFrame(categorical_fea)
    print(cat_df.shape)
    fea_df = pd.concat([num_df, cat_df], axis=1)
    
    fea_df.columns = num_cols + cat_cols
    if is_train:
        fea_df['label'] = label
    print(fea_df.shape)
    return fea_df

test_fea = feature_engineering(test_df, is_train=False)
train_fea = feature_engineering(train_df)

  0%|          | 6/146135 [00:00<40:39, 59.90it/s]

(228, 362)
(228, 20)
(228, 382)


100%|██████████| 146135/146135 [1:36:13<00:00, 25.31it/s]  


(146135, 362)
(146135, 20)
(146135, 383)
CPU times: user 1h 36min 31s, sys: 26.4 s, total: 1h 36min 57s
Wall time: 1h 36min 29s


In [6]:
train_fea.to_pickle('data/train_fea.pkl')
test_fea.to_pickle('data/test_fea.pkl')

In [9]:
train_fea.head()

,num_0,num_1,num_2,num_3,num_4,num_5,num_6,num_7,num_8,num_9,num_10,num_11,num_12,num_13,num_14,num_15,num_16,num_17,num_18,num_19,num_20,num_21,num_22,num_23,num_24,num_25,num_26,num_27,num_28,num_29,num_30,num_31,num_32,num_33,num_34,num_35,num_36,num_37,num_38,num_39,num_40,num_41,num_42,num_43,num_44,num_45,num_46,num_47,num_48,num_49,num_50,num_51,num_52,num_53,num_54,num_55,num_56,num_57,num_58,num_59,num_60,num_61,num_62,num_63,num_64,num_65,num_66,num_67,num_68,num_69,num_70,num_71,num_72,num_73,num_74,num_75,num_76,num_77,num_78,num_79,num_80,num_81,num_82,num_83,num_84,num_85,num_86,num_87,num_88,num_89,num_90,num_91,num_92,num_93,num_94,num_95,num_96,num_97,num_98,num_99,num_100,num_101,num_102,num_103,num_104,num_105,num_106,num_107,num_108,num_109,num_110,num_111,num_112,num_113,num_114,num_115,num_116,num_117,num_118,num_119,num_120,num_121,num_122,num_123,num_124,num_125,num_126,num_127,num_128,num_129,num_130,num_131,num_132,num_133,num_134,num_135,num_136,num_137,num_138,num_139,num_140,num_141,num_142,num_143,num_144,num_145,num_146,num_147,num_148,num_149,num_150,num_151,num_152,num_153,num_154,num_155,num_156,num_157,num_158,num_159,num_160,num_161,num_162,num_163,num_164,num_165,num_166,num_167,num_168,num_169,num_170,num_171,num_172,num_173,num_174,num_175,num_176,num_177,num_178,num_179,num_180,num_181,num_182,num_183,num_184,num_185,num_186,num_187,num_188,num_189,num_190,num_191,num_192,num_193,num_194,num_195,num_196,num_197,num_198,num_199,num_200,num_201,num_202,num_203,num_204,num_205,num_206,num_207,num_208,num_209,num_210,num_211,num_212,num_213,num_214,num_215,num_216,num_217,num_218,num_219,num_220,num_221,num_222,num_223,num_224,num_225,num_226,num_227,num_228,num_229,num_230,num_231,num_232,num_233,num_234,num_235,num_236,num_237,num_238,num_239,num_240,num_241,num_242,num_243,num_244,num_245,num_246,num_247,num_248,num_249,num_250,num_251,num_252,num_253,num_254,num_255,num_256,num_257,num_258,num_259,num_260,num_261,num_262,num_263,num_264,num_265,num_266,num_267,num_268,num_269,num_270,num_271,num_272,num_273,num_274,num_275,num_276,num_277,num_278,num_279,num_280,num_281,num_282,num_283,num_284,num_285,num_286,num_287,num_288,num_289,num_290,num_291,num_292,num_293,num_294,num_295,num_296,num_297,num_298,num_299,num_300,num_301,num_302,num_303,num_304,num_305,num_306,num_307,num_308,num_309,num_310,num_311,num_312,num_313,num_314,num_315,num_316,num_317,num_318,num_319,num_320,num_321,num_322,num_323,num_324,num_325,num_326,num_327,num_328,num_329,num_330,num_331,num_332,num_333,num_334,num_335,num_336,num_337,num_338,num_339,num_340,num_341,num_342,num_343,num_344,num_345,num_346,num_347,num_348,num_349,num_350,num_351,num_352,num_353,num_354,num_355,num_356,num_357,num_358,num_359,num_360,num_361,cat_0,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18,cat_19,label
0,0.424361,0.703056,0.0,0.224311,0.703056,4.243611,0.526111,9.0,10.0,0.9,1.0,-0.583415,-1.031987,0.0,0.070306,0.175000,0.0,0.068578,0.175000,0.703056,0.043056,9.0,10.0,0.9,1.0,0.664976,-1.296856,0.0,113.891127,113.903790,113.883643,0.005706,0.020147,1138.911267,113.891168,10.0,10.0,1.0,1.0,0.794366,0.019517,113.883643,22.398996,22.437238,22.368845,0.019567,0.068393,223.989957,22.393654,10.0,10.0,1.0,1.0,0.411714,-0.646674,22.437238,12.3,22.0,7.0,5.080354,15.0,123.0,10.5,10.0,7.0,1.0,0.7,0.936854,-0.539103,11.0,16360.0,19100.0,12900.0,1456.845908,6200.0,163600.0,16550.0,10.0,8.0,1.0,0.8,-0.680156,1.452750,16700.0,0.162923,0.194920,0.123945,0.020220,0.070975,1.629230,0.168173,10.0,10.0,1.0,1.0,-0.354575,-0.661659,0.123945,0.839575,1.629230,0.123945,0.481059,1.505284,8.395749,0.819407,10.0,10.0,1.0,1.0,0.117177,-1.217487,0.123945,0.007097,0.019106,0.0,0.006184,0.019106,0.070975,0.006781,9.0,10.0,0.9,1.0,0.472278,-0.974732,0.0,58.899778,58.913775,58.890607,0.006525,0.023168,588.997777,58.900053,10.0,10.0,1.0,1.0,0.639501,-0.183323,58.890607,323.930840,588.997777,58

In [10]:
use_cols = []
# imp = pd.read_pickle("imp.pkl")
# del_cols = imp['feat'].values.tolist()[-50:]

for col in ['num_{}'.format(str(i)) for i in range(362)]:
    if np.std(train_fea[col].values) > 0.01:
        use_cols.append(col)
print(len(use_cols))

338


In [11]:
# label_encoder
from sklearn.preprocessing import LabelEncoder

test_fea['label'] = -1e8
data_fea = pd.concat([train_fea, test_fea])
data_fea = data_fea.reset_index(drop=True)

for col in ['cat_{}'.format(str(i)) for i in range(20)]:
    print(col)
    le = LabelEncoder()
    data_fea[col] = le.fit_transform(data_fea[col])
    data_fea[col] = data_fea[col].astype('category')

train_feas = data_fea[data_fea['label'] != -1e8].reset_index(drop=True)
test_feas = data_fea[data_fea['label'] == -1e8].reset_index(drop=True)
del test_feas['label']
print(train_feas.shape, test_feas.shape)

all_feas = use_cols + ['cat_{}'.format(str(i)) for i in range(20)]
cat_feas = ['cat_{}'.format(str(i)) for i in range(20)]
print(len(all_feas), len(cat_feas))

cat_0
cat_1
cat_2
cat_3
cat_4
cat_5
cat_6
cat_7
cat_8
cat_9
cat_10
cat_11
cat_12
cat_13
cat_14
cat_15
cat_16
cat_17
cat_18
cat_19
(146135, 383) (228, 382)
358 20


In [15]:
from sklearn.metrics import mean_squared_error as mse 
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold

In [14]:
# !pip install lightgbm

In [16]:
def mse_score_eval(preds, valid):
    labels = valid.get_label()
    scores = mse(y_true=labels, y_pred=preds)
    return 'mse_score', scores, False

def build_model(train, test, label, seed=2020, is_shuffle=True):
    imp = pd.DataFrame()  # 特征重要性
    imp['feat'] = all_feas

    train_pred = np.zeros((train.shape[0], ))
    test_pred = np.zeros((test.shape[0], ))
    n_splits = 10
    # Kfold
    # fold = GroupKFold(n_splits=n_splits)
    fold = KFold(n_splits=n_splits, shuffle=is_shuffle, random_state=seed)
    # groups = train_df['loadingOrder'].values
    kf_way = fold.split(train)
    # params
    params = {
        'learning_rate': 0.05,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'num_leaves': 128,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.75,
        'bagging_freq': 5,
        'seed': 8,
        'bagging_seed': 1,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 20,
        'nthread': -1,
        'verbose': 1,
    }
    # train
    for n_fold, (train_idx, valid_idx) in enumerate(kf_way, start=1):
        print("fold ", n_fold)
        train_x, train_y = train.iloc[train_idx], label[train_idx]
        valid_x, valid_y = train.iloc[valid_idx], label[valid_idx]
        # 数据加载
        n_train = lgb.Dataset(train_x, label=train_y)
        n_valid = lgb.Dataset(valid_x, label=valid_y)

        clf = lgb.train(
            params=params,
            train_set=n_train,
            categorical_feature=cat_feas,
            num_boost_round=10000,
            valid_sets=[n_train, n_valid],
            early_stopping_rounds=50,
            verbose_eval=100,
            feval=mse_score_eval
        )
        train_pred[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration)
        test_pred += clf.predict(test, num_iteration=clf.best_iteration)/fold.n_splits

        imp['gain' + str(n_fold + 1)] = clf.feature_importance(importance_type='gain')
        imp['split' + str(n_fold + 1)] = clf.feature_importance(importance_type='split')

    print("train mse: ", mse(label, train_pred))
    result = pd.DataFrame({
        'loadingOrder': test_df['loadingOrder'], 
        'label': test_pred,
    })
    return result, imp

result, imp = build_model(train_feas[all_feas], test_feas[all_feas], train_feas['label'].values, is_shuffle=True)

fold  1


/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_0', 'cat_1', 'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17', 'cat_18', 'cat_19', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 50 rounds
[100]	training's l2: 345.008	training's mse_score: 345.008	valid_1's l2: 741.237	valid_1's mse_score: 741.237
[200]	training's l2: 129.463	training's mse_score: 129.463	valid_1's l2: 509.409	valid_1's mse_score: 509.409
[300]	training's l2: 72.1607	training's mse_score: 72.1607	valid_1's l2: 445.352	valid_1's mse_score: 445.352
[400]	training's l2: 45.6524	training's mse_score: 45.6524	valid_1's l2: 413.386	valid_1's mse_score: 413.386
[500]	training's l2: 30.953	training's mse_score: 30.953	valid_1's l2: 394.182	valid_1's mse_score: 394.182
[600]	training's l2: 21.9024	training's mse_score: 21.9024	valid_1's l2: 381.571	valid_1's mse_score: 381.571
[700]	training's l2: 16.2897	training's mse_score: 16.2897	valid_1's l2: 373.756	valid_1's mse_score: 373.756
[800]	training's l2: 12.3894	training's mse_score: 12.3894	valid_1's l2: 368.148	valid_1's mse_score: 368.148
[900]	training's l2: 9.70774	training's mse_score: 9.70774	va

/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_0', 'cat_1', 'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17', 'cat_18', 'cat_19', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 50 rounds
[100]	training's l2: 343.027	training's mse_score: 343.027	valid_1's l2: 636.608	valid_1's mse_score: 636.608
[200]	training's l2: 127.677	training's mse_score: 127.677	valid_1's l2: 429.565	valid_1's mse_score: 429.565
[300]	training's l2: 70.2632	training's mse_score: 70.2632	valid_1's l2: 369.95	valid_1's mse_score: 369.95
[400]	training's l2: 44.3122	training's mse_score: 44.3122	valid_1's l2: 340.874	valid_1's mse_score: 340.874
[500]	training's l2: 30.3029	training's mse_score: 30.3029	valid_1's l2: 322.832	valid_1's mse_score: 322.832
[600]	training's l2: 21.4444	training's mse_score: 21.4444	valid_1's l2: 310.887	valid_1's mse_score: 310.887
[700]	training's l2: 15.9209	training's mse_score: 15.9209	valid_1's l2: 302.568	valid_1's mse_score: 302.568
[800]	training's l2: 11.9572	training's mse_score: 11.9572	valid_1's l2: 296.47	valid_1's mse_score: 296.47
[900]	training's l2: 9.23305	training's mse_score: 9.23305	vali

...

Training until validation scores don't improve for 50 rounds
[100]	training's l2: 337.92	training's mse_score: 337.92	valid_1's l2: 655.822	valid_1's mse_score: 655.822
[200]	training's l2: 125.207	training's mse_score: 125.207	valid_1's l2: 426.246	valid_1's mse_score: 426.246
[300]	training's l2: 69.002	training's mse_score: 69.002	valid_1's l2: 364.581	valid_1's mse_score: 364.581
[400]	training's l2: 43.3076	training's mse_score: 43.3076	valid_1's l2: 334.532	valid_1's mse_score: 334.532
[500]	training's l2: 29.1311	training's mse_score: 29.1311	valid_1's l2: 316.593	valid_1's mse_score: 316.593
[600]	training's l2: 20.5512	training's mse_score: 20.5512	valid_1's l2: 305.203	valid_1's mse_score: 305.203
[700]	training's l2: 15.2334	training's mse_score: 15.2334	valid_1's l2: 297.73	valid_1's mse_score: 297.73
[800]	training's l2: 11.5289	training's mse_score: 11.5289	valid_1's l2: 292.156	valid_1's mse_score: 292.156
[900]	training's l2: 8.91058	training's mse_score: 8.91058	valid_

/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_0', 'cat_1', 'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17', 'cat_18', 'cat_19', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 50 rounds
[100]	training's l2: 347.167	training's mse_score: 347.167	valid_1's l2: 531.646	valid_1's mse_score: 531.646
[200]	training's l2: 128.309	training's mse_score: 128.309	valid_1's l2: 332.822	valid_1's mse_score: 332.822
[300]	training's l2: 70.6322	training's mse_score: 70.6322	valid_1's l2: 280.774	valid_1's mse_score: 280.774
[400]	training's l2: 44.4209	training's mse_score: 44.4209	valid_1's l2: 254.197	valid_1's mse_score: 254.197
[500]	training's l2: 30.0037	training's mse_score: 30.0037	valid_1's l2: 239.548	valid_1's mse_score: 239.548
[600]	training's l2: 21.001	training's mse_score: 21.001	valid_1's l2: 229.626	valid_1's mse_score: 229.626
[700]	training's l2: 15.4838	training's mse_score: 15.4838	valid_1's l2: 223.477	valid_1's mse_score: 223.477
[800]	training's l2: 11.7586	training's mse_score: 11.7586	valid_1's l2: 218.792	valid_1's mse_score: 218.792
[900]	training's l2: 9.15069	training's mse_score: 9.15069	va

/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_0', 'cat_1', 'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17', 'cat_18', 'cat_19', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 50 rounds
[100]	training's l2: 340.511	training's mse_score: 340.511	valid_1's l2: 746.031	valid_1's mse_score: 746.031
[200]	training's l2: 127.125	training's mse_score: 127.125	valid_1's l2: 510.166	valid_1's mse_score: 510.166
[300]	training's l2: 69.5334	training's mse_score: 69.5334	valid_1's l2: 444.792	valid_1's mse_score: 444.792
[400]	training's l2: 43.5897	training's mse_score: 43.5897	valid_1's l2: 413.319	valid_1's mse_score: 413.319
[500]	training's l2: 29.349	training's mse_score: 29.349	valid_1's l2: 395.485	valid_1's mse_score: 395.485
[600]	training's l2: 20.6591	training's mse_score: 20.6591	valid_1's l2: 383.79	valid_1's mse_score: 383.79
[700]	training's l2: 15.1698	training's mse_score: 15.1698	valid_1's l2: 375.985	valid_1's mse_score: 375.985
[800]	training's l2: 11.4515	training's mse_score: 11.4515	valid_1's l2: 370.687	valid_1's mse_score: 370.687
[900]	training's l2: 8.91971	training's mse_score: 8.91971	vali

In [18]:
imp['gain'] = imp[[f for f in imp.columns if 'gain' in f]].sum(axis=1)/10
imp['split'] = imp[[f for f in imp.columns if 'split' in f]].sum(axis=1)/10

imp = imp[['feat', 'gain', 'split']]
imp

,feat,gain,split
0,num_0,7.003265e+06,820.4
1,num_1,4.160235e+07,1196.6
2,num_3,1.261791e+07,964.4
3,num_4,9.811510e+06,176.0
4,num_5,2.397947e+06,664.8
5,num_6,5.525129e+06,1222.4
6,num_7,3.762070e+05,170.4
7,num_8,1.251110e+05,39.6
8,num_9,7.263034e+04,9.6
9,num_11,5.326924e+06,2567.2


In [19]:
imp = imp.sort_values(by=['gain'], ascending=False)
imp = imp.reset_index(drop=True)
imp.to_pickle("imp.pkl")

In [20]:
imp

,feat,gain,split
0,num_212,2.543955e+10,1132.8
1,num_353,5.001448e+09,647.6
2,num_352,2.151259e+09,669.6
3,cat_6,1.363523e+09,4315.6
4,num_128,1.292548e+09,1136.0
5,cat_5,9.839232e+08,2939.7
6,cat_7,8.565134e+08,8352.6
7,cat_1,5.515756e+08,49841.1
8,cat_11,1.505897e+08,26706.0
9,num_360,1.379528e+08,709.6


In [21]:
# train mse:  1174.767318829244
# train mse:  273.7163487202298

sub_FORMAT = "%Y/%m%d  %H:%M:%S"
UTC_FORMAT = "%Y-%m-%dT%H:%M:%S.%fZ"

def get_time(x, y):
    x = x.split(' ')[-1]
    x = datetime.datetime.strptime(x, UTC_FORMAT)
    day = y // 24 
    seconds = int((y - (y//24)*24)*3600)
    res = x + datetime.timedelta(days=day, seconds=seconds)
    str_res = res.strftime(sub_FORMAT)
    return res 

# result = pd.DataFrame({
#         'loadingOrder': test_loadingOrder, 
#         'label': test_pred,
#     })

result['timestamp'] = test_df['timestamp']
result['ETA'] = list(map(lambda x, y: get_time(x, y), result['timestamp'], result['label']))

In [22]:
test_data = pd.read_csv("data/B_testData0626.csv") 
def get_data(data, mode='train'):
    
    assert mode=='train' or mode=='test'
#     if mode=='train':
#         data['vesselNextportETA'] = pd.to_datetime(data['vesselNextportETA'], infer_datetime_format=True) # 转换时间
    if mode=='test':
        data['temp_timestamp'] = data['timestamp']
        data['onboardDate'] = pd.to_datetime(data['onboardDate'], infer_datetime_format=True)
    
    data['timestamp'] = pd.to_datetime(data['timestamp'], infer_datetime_format=True)
    return data

test_data = get_data(test_data, mode='test')

test_data.sort_values(['loadingOrder', 'timestamp'], inplace=True)
test_data = test_data.reset_index(drop=True) 

test_data = test_data.merge(result[['loadingOrder', 'ETA']], on='loadingOrder', how='left')
test_data['ETA'] = test_data['ETA'].apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))
test_data.drop(['direction','TRANSPORT_TRACE'],axis=1,inplace=True)
test_data['onboardDate'] = test_data['onboardDate'].apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))
test_data['creatDate'] = pd.datetime.now().strftime('%Y/%m/%d  %H:%M:%S')
test_data['timestamp'] = test_data['temp_timestamp']
# 整理columns顺序
result = test_data[['loadingOrder', 'timestamp', 'longitude', 'latitude', 'carrierName', 'vesselMMSI', 'onboardDate', 'ETA', 'creatDate']]

/opt/conda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:23: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.


In [23]:
result

,loadingOrder,timestamp,longitude,latitude,carrierName,vesselMMSI,onboardDate,ETA,creatDate
0,AE223035353902,2019-07-03T21:16:48.000Z,120.093858,22.581320,OIEQNT,C2075927370,2019/07/02 04:12:48,2019/07/26 03:43:57,2020/06/27 08:10:30
1,AE223035353902,2019-07-03T21:34:48.000Z,120.035707,22.617522,OIEQNT,C2075927370,2019/07/02 04:12:48,2019/07/26 03:43:57,2020/06/27 08:10:30
2,AE223035353902,2019-07-03T21:51:18.000Z,119.981800,22.658465,OIEQNT,C2075927370,2019/07/02 04:12:48,2019/07/26 03:43:57,2020/06/27 08:10:30
3,AE223035353902,2019-07-03T21:54:18.000Z,119.970845,22.668688,OIEQNT,C2075927370,2019/07/02 04:12:48,2019/07/26 03:43:57,2020/06/27 08:10:30
4,AE223035353902,2019-07-03T22:11:08.000Z,119.953628,22.756897,OIEQNT,C2075927370,2019/07/02 04:12:48,2019/07/26 03:43:57,2020/06/27 08:10:30
...,...,...,...,...,...,...,...,...,...
34707,ZZ524449869421,2020-03-17T04:02:38.000Z,103.776707,1.252897,BHSOUA,P2595193878,2020/03/13 06:07:28,2020/04/01 04:40:59,2020/06/27 08:10:30
34708,ZZ524449869421,2020-03-17T04:03:18.000Z,103.776312,1.253418,BHSOUA,P2595193878,2020/03/13 06:07:28,2020/04/01 04:40:59,2020/06/27 08:10:30
34709,ZZ524449869421,2020-03-17T04:05:18.000Z,103.775175,1.254865,BHSOUA,P2595193878,2020/03/13 06:07:28,2020/04/01 04:40:59,2020/06/27 08:10:30
34710,ZZ524449869421,2020-03-17T04:05:58.000Z,103.774803,1.255285,BHSOUA,P2595193878,2020/03/13 06:07:28,2020/04/01 04:40:59,2020/06/27 08:10:30


In [24]:
result.to_csv('data/B_lgb_10fold.csv', index=False)